In [ ]:
#IMPORTS

import math
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import sklearn
from sklearn.metrics import f1_score
import seaborn as sns
import tensorflow as tf
import sklearn.metrics as metrics
from sklearn import tree
from sklearn.svm import SVC
from tensorflow import keras
from keras.models import Sequential
from keras.callbacks import History
from keras.layers import Dense
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree, export_text
from sklearn.pipeline import Pipeline
from sklearn import decomposition, datasets, linear_model
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, precision_recall_curve

In [ ]:
# df
df = pd.read_csv(r"f1dataset2.csv", encoding='utf-8')

# shuffle data
df.sample(frac=1, random_state=42)  # Set random_state for reproducibility

In [ ]:
label_encode = LabelEncoder()
df['relativecompound'] = label_encode.fit_transform(df['relativecompound']) #hard = 0 , med = 1 , soft = 2

In [ ]:
# Separate categorical and numerical features
cat_features = ['remaining_pit_stops', 'location', 'fulfilled_second_compound', 'number_of_available_compounds']
num_features = ['race_progress']

# Splitting the data into input features (X) and pit stop labels (y)
x_cat = df[cat_features]
x_num = df[num_features]
X = pd.concat([x_cat, x_num], axis=1)
y = df['relativecompound']

# One-hot encoding categorical features
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(x_cat).toarray()

# Combining encoded categorical features with numerical features
X = np.concatenate((X_encoded, x_num), axis=1)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## SVM

In [ ]:
# Train the SVM classifier
svm = SVC(kernel='linear')
svm.fit(X_train_scaled, y_train)

# Make predictions
y_pred = svm.predict(X_test_scaled)

In [ ]:
f1_svm = f1_score(y_test, y_pred, average='weighted')
print("F1 score:", f1_svm*100)

## DT

In [ ]:
# Train the decision tree classifier
tree = DecisionTreeClassifier()
tree.fit(X_train_scaled, y_train)

# Make predictions
y_pred = tree.predict(X_test_scaled)

In [ ]:
f1_dt = f1_score(y_test, y_pred, average='weighted')
print("F1 score:", f1_dt*100)

## LR

In [ ]:
# Train the logistic regression model with increased max_iter
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_scaled, y_train)

# Make predictions
y_pred = lr.predict(X_test_scaled)

In [ ]:
f1_lr = f1_score(y_test, y_pred, average='weighted')
print("F1 score:", f1_lr * 100)

## NN

In [ ]:
# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder on y_train and transform y_train and y_test
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Determine the number of classes (unique labels)
num_classes = len(label_encoder.classes_)

# Define the neural network architecture
model = Sequential()
model.add(Dense(16, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Sigmoid activation for binary classification

# Compile the model with binary cross-entropy loss
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model using the encoded labels
model.fit(X_train_scaled, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test_encoded))


In [ ]:
# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred_labels = np.argmax(y_pred_prob, axis=1)  

# Calculate the F1 score
f1_nn = f1_score(y_test_encoded, y_pred_labels, average='weighted')
print("F1 score:", f1_nn * 100)